# CIFAR-10 Challange - 김영인

1. resnet18
2. \+ Data Augmentation(RandomResizedCrop, RandomHorizontalFlip)
3. \+ Data Normalize
4. \+ regularization(weight decay)
5. \+ Optimizer(momentum, scheduler)

In [1]:
import random

import torch
import numpy as np
from matplotlib import pyplot as plt

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models, datasets

random_seed = 4332
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

device0 = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device1 = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

device = device0
print(f"device: {device}") if torch.cuda.is_available() else print("device: cpu")

device: cuda:0


## Hyper parameter Setting 

In [2]:
learning_rate = 1e-2
training_epochs = 50
batch_size = 64
momentum = 0.9
scheduler_step = 1
scheduler_gamma = 0.9

## Load & Preprocess Data

In [3]:
transform_train = transforms.Compose(
    [transforms.RandomResizedCrop(224), # data augmentation, 224: image size, ImageNet pretrained model에 맞추기 위해서 224 size로 설정
     transforms.RandomHorizontalFlip(), # data augmentation, 좌우로 대칭
     transforms.ToTensor(), # numpy array를 pytorch tensor로 바꿔주는 역할
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)) # dataset의 mean, std를 이용해서 -1~1 로 normalize
    ])

transform_test = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = datasets.CIFAR10(root='./data', train=True,
                                      download=True, transform=transforms.ToTensor())
trainloader = DataLoader(trainset, batch_size=batch_size,
                                         shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./data', train=False,
                                      download=True, transform=transforms.ToTensor())
testloader = DataLoader(testset, batch_size=4,
                                        shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## Load and Define pretrained model and fine-tuning

In [4]:
class ResNet18(nn.Module):
    def __init__ (self):
        super(ResNet18, self).__init__()
        self.resnet = models.resnet18(pretrained=True)
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_ftrs, 10)
        
    def forward(self, x):
        x = self.resnet(x) # batch_size * 10
        return x

## Load Model

In [5]:
net = ResNet18()
net = net.to(device)

criterion = nn.CrossEntropyLoss() 
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)
scheduler = optim.lr_scheduler.StepLR(optimizer, 
                                      step_size = scheduler_step,
                                      gamma = scheduler_gamma)

## Train

In [6]:
net.train()

for epoch in range(training_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()

        outputs = net(inputs) 
        loss =  criterion(outputs, labels)
        loss.backward()
        
        optimizer.step()
        
        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
        
print('Finished Training')

[1,   100] loss: 1.600
[1,   200] loss: 1.163
[1,   300] loss: 1.030
[1,   400] loss: 0.990
[1,   500] loss: 0.895
[1,   600] loss: 0.839
[1,   700] loss: 0.823
[2,   100] loss: 0.732
[2,   200] loss: 0.638
[2,   300] loss: 0.715
[2,   400] loss: 0.621
[2,   500] loss: 0.633
[2,   600] loss: 0.628
[2,   700] loss: 0.598
[3,   100] loss: 0.488
[3,   200] loss: 0.478
[3,   300] loss: 0.519
[3,   400] loss: 0.505
[3,   500] loss: 0.511
[3,   600] loss: 0.508
[3,   700] loss: 0.501
[4,   100] loss: 0.372
[4,   200] loss: 0.386
[4,   300] loss: 0.379
[4,   400] loss: 0.387
[4,   500] loss: 0.402
[4,   600] loss: 0.412
[4,   700] loss: 0.422
[5,   100] loss: 0.305
[5,   200] loss: 0.295
[5,   300] loss: 0.303
[5,   400] loss: 0.303
[5,   500] loss: 0.319
[5,   600] loss: 0.333
[5,   700] loss: 0.340
[6,   100] loss: 0.224
[6,   200] loss: 0.236
[6,   300] loss: 0.246
[6,   400] loss: 0.278
[6,   500] loss: 0.280
[6,   600] loss: 0.271
[6,   700] loss: 0.285
[7,   100] loss: 0.189
[7,   200] 

[50,   200] loss: 0.002
[50,   300] loss: 0.007
[50,   400] loss: 0.008
[50,   500] loss: 0.008
[50,   600] loss: 0.013
[50,   700] loss: 0.016
Finished Training


## Test

In [7]:
class_correct = np.zeros(10)
class_total = np.zeros(10)

net.eval()

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2f %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))
print('Mean Accuracy : ', 100*np.sum(class_correct)/np.sum(class_total))

Accuracy of plane : 83.000000 %
Accuracy of   car : 93.100000 %
Accuracy of  bird : 76.700000 %
Accuracy of   cat : 60.500000 %
Accuracy of  deer : 78.800000 %
Accuracy of   dog : 72.400000 %
Accuracy of  frog : 91.700000 %
Accuracy of horse : 84.300000 %
Accuracy of  ship : 90.600000 %
Accuracy of truck : 77.000000 %
Mean Accuracy :  80.81
